In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
import numpy as np
data = pd.read_csv("/kaggle/input/anya-bondman/trainData.csv") #Train Dataset
data_t = pd.read_csv("/kaggle/input/anya-bondman/testData.csv") #Test Dataset

In [ ]:
data_t.head()

In [ ]:
data.shape

In [ ]:
data[data['is_fraud']==0].shape[0]


In [ ]:
data[data['is_fraud']==1].shape[0]

In [ ]:
data_t.columns

In [ ]:
data.describe()

In [ ]:
data.info()

# EDA

In [ ]:
data.isnull().sum()

In [ ]:
data.head()

In [ ]:
data = data.drop("trans_num", axis=1)


In [ ]:
data.info()


In [ ]:
data.cc_num.value_counts()

In [ ]:
data = data.drop(columns=["first","last"])

In [ ]:
data.info()

In [ ]:
data["recency"] = data.groupby(by="cc_num")["unix_time"].diff()
data

In [ ]:
data["recency"].isnull().sum()

In [ ]:
data.loc[data.recency.isnull(),["recency"]] = -1

In [ ]:
data.isnull().sum()

In [ ]:
# converting trans_date_trans_time to datetime

data["trans_date_trans_time"] = pd.to_datetime(data["trans_date_trans_time"])

In [ ]:
data.info()

In [ ]:
data[data["cc_num"] == 4873783502705038]


In [ ]:
data= data.drop("unix_time",axis=1)

In [ ]:
data.info()

In [ ]:
data= data.drop("id",axis=1)

In [ ]:
data.info()

In [ ]:
# checking do column
data.dob.value_counts()

In [ ]:
# in predicting customer frauds, dob doesn't play any role that whether there will be a fraud with customer or not, so removing dob column too
data = data.drop("dob",axis=1)

# checking columns now

data.info()

In [ ]:
# sometimes distance from the customer's home location to the merchant's location can prove out to be main reason for fraud, so taking the 
# difference of longitude and lattitude of respective columns
data["lat_diff"] = abs(data.lat - data.merch_lat)
data["long_diff"] = abs(data["long"] - data["merch_long"])


In [ ]:
# checking data
data.head()

In [ ]:

# Calculate displacement using lat_diff and long_diff
data["displacement"] = np.sqrt(pow((data["lat_diff"] * 110), 2) + pow((data["long_diff"] * 110), 2))


In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
# now since we got the displacement so longitudes and lattitudes columns are of no use now, so we can remove them
data = data.drop(columns = ["lat","long","merch_lat","merch_long","lat_diff","long_diff"])

In [ ]:
data.info()

In [ ]:
data= data.drop(columns = ["city","zip","street"])
data.info()

In [ ]:
data.displacement.describe()

In [ ]:
data.loc[(data["displacement"]<45),["location"]] = "Nearby"
data.loc[((data["displacement"]>45) & (data["displacement"]<90)),["location"]] = "Far Away"
data.loc[(data["displacement"]>90),["location"]] = "Long Distance"
data.info()

In [ ]:
data.location.value_counts(normalize = True)

In [ ]:
data["Time"] = pd.to_datetime(data["trans_date_trans_time"],"%H:%M").dt.time

In [ ]:
data.head()

In [ ]:
data["Time"] = pd.to_datetime(data["trans_date_trans_time"]).dt.hour

In [ ]:
data["Time"] = pd.to_datetime(data["trans_date_trans_time"]).dt.hour
# segregating city_population tab on the basis of less dense, adequately densed, densely populated
data.loc[(data["city_pop"]<10000),["city_pop_segment"]] = "Less Dense"
data.loc[((data["city_pop"]>10000) & (data["city_pop"]<50000)),["city_pop_segment"]] = "Adequately Dense"
data.loc[(data["city_pop"]>50000),["city_pop_segment"]] = "Densely populated"

In [ ]:
data.city_pop_segment.value_counts(normalize = True)

In [ ]:
data = data.drop("city_pop",axis=1)

In [ ]:
data.recency = data.recency.apply(lambda x: float((x/60)/60))
# dividing recency to segments based on number of hours passed
data.loc[(data["recency"]<1),["recency_segment"]] = "Recent_Transaction"
data.loc[((data["recency"]>1) & (data["recency"]<6)),["recency_segment"]] = "Within 6 hours"
data.loc[((data["recency"]>6) & (data["recency"]<12)),["recency_segment"]] = "After 6 hours"
data.loc[((data["recency"]>12) & (data["recency"]<24)),["recency_segment"]] = "After Half-Day"
data.loc[(data["recency"]>24),["recency_segment"]] = "After 24 hours"
data.loc[(data["recency"]<0),["recency_segment"]] = "First Transaction"
data.recency_segment.value_counts(normalize = True)

In [ ]:
# examining category column
data.category.value_counts()

In [ ]:
# examining merchant column
data.merchant.value_counts()

In [ ]:
data.columns


# Work on test data Column

In [ ]:
# we cannot work on trans_num as there is no unique pattern, so dropping it
data_t = data_t.drop("trans_num",axis=1)

In [ ]:
data_t = data_t.drop(columns=["first","last"])

In [ ]:
data_t["recency"] = data_t.groupby(by="cc_num")["unix_time"].diff()

In [ ]:
# we are getting null values because as 983 because there are 983 unique values of cards, this means whenever the cc_num group changes
# python makes the first value of every group null, so making them as starting payment, we will initialize null values to -1

data_t.loc[data_t.recency.isnull(),["recency"]] = -1

In [ ]:
# converting trans_date_trans_time to datetime

data_t["trans_date_trans_time"] = pd.to_datetime(data_t["trans_date_trans_time"])

In [ ]:
# we now don't need unix_time so we can drop it
data_t= data_t.drop("unix_time",axis=1)

In [ ]:

data_t = data_t.drop("id",axis=1)

In [ ]:
data_t = data_t.drop("dob",axis=1)

In [ ]:
data_t["lat_diff"] = abs(data_t.lat - data_t.merch_lat)
data_t["long_diff"] = abs(data_t["long"] - data_t["merch_long"])
data_t["displacement"] = np.sqrt(pow((data_t["lat_diff"]*110),2) + pow((data_t["long_diff"]*110),2))

In [ ]:
data_t = data_t.drop(columns = ["lat","long","merch_lat","merch_long","lat_diff","long_diff"])

In [ ]:
# since state contains both city and zip code and street comes under city, so we can move with state column and drop street, city and zip
# we can work with cities through their population parameter, as names of cities cannot implement whether a fraud will be done or not, while
# population of a city can.
data_t= data_t.drop(columns = ["city","zip","street"])

In [ ]:
# now we can bin the displacement into near, far and very far records
# if merchant lies between the range of 0-45 then it is near, while above 45 but below 90 will be far and rest can be very far

data_t.loc[(data_t["displacement"]<45),["location"]] = "Nearby"
data_t.loc[((data_t["displacement"]>45) & (data_t["displacement"]<90)),["location"]] = "Far Away"
data_t.loc[(data_t["displacement"]>90),["location"]] = "Long Distance"

In [ ]:
# Although date part in column trans_date_trans_time is not important because that is historical data, but time part of the component is important
# so creating a column of time

data_t["Time"] = pd.to_datetime(data_t["trans_date_trans_time"],"%H:%M").dt.time
# converting Time column to datetime
data_t["Time"] = pd.to_datetime(data_t["trans_date_trans_time"]).dt.hour

In [ ]:
data_t.loc[(data_t["city_pop"]<10000),["city_pop_segment"]] = "Less Dense"
data_t.loc[((data_t["city_pop"]>10000) & (data_t["city_pop"]<50000)),["city_pop_segment"]] = "Adequately Dense"
data_t.loc[(data_t["city_pop"]>50000),["city_pop_segment"]] = "Densely populated"

In [ ]:
# dropping column city_pop as it is of no use now
data_t = data_t.drop("city_pop",axis=1)

In [ ]:
data_t.recency = data_t.recency.apply(lambda x: float((x/60)/60))

In [ ]:
data_t.loc[(data_t["recency"]<1),["recency_segment"]] = "Recent_Transaction"
data_t.loc[((data_t["recency"]>1) & (data_t["recency"]<6)),["recency_segment"]] = "Within 6 hours"
data_t.loc[((data_t["recency"]>6) & (data_t["recency"]<12)),["recency_segment"]] = "After 6 hours"
data_t.loc[((data_t["recency"]>12) & (data_t["recency"]<24)),["recency_segment"]] = "After Half-Day"
data_t.loc[(data_t["recency"]>24),["recency_segment"]] = "After 24 hours"
data_t.loc[(data_t["recency"]<0),["recency_segment"]] = "First Transaction"

In [ ]:
data_t.columns

# Univariate Analysis

In [ ]:
# checking the peak hours for transaction
data.info()

In [ ]:
data.head()

In [ ]:
# let's initialize a separate data containing fraud transactionsto analyze trends
data_fraud = data[data["is_fraud"]==1]

In [ ]:
sns.kdeplot(data_fraud["Time"])
plt.xlim(left = 0,right = 24)
plt.show()

In [ ]:
# lets have a look about the fraud transactions done in cities according to their populations
sns.histplot(data_fraud["city_pop_segment"])
plt.show()


In [ ]:
# let's have a look on state-wise fraud transactions
plt.figure(figsize = [8,10])
sns.countplot(y=data_fraud.state)
plt.show()

In [ ]:
sns.kdeplot(data_fraud.trans_date_trans_time)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
sns.countplot(y=data_fraud.recency_segment)
plt.show()

In [ ]:
sns.countplot(y=data.recency_segment)
plt.show()

In [ ]:
data["recency_segment"].value_counts()

In [ ]:
# checking how recent the transaction is can be termed as fraud
sns.kdeplot(data_fraud.recency)
#plt.xlim(right = 15000,left = -1)
plt.show()

In [ ]:
sns.kdeplot(data_fraud[data_fraud["recency_segment"] == "Recent_Transaction"].Time)
plt.xlim(left=0,right=24)
plt.show()

In [ ]:
# checking separately for first transactions done through credit cards.
(data_fraud[data["recency_segment"] == "First Transaction"]["is_fraud"].sum()/len(data[data["recency"]<0]["is_fraud"]))*100

In [ ]:
data_fraud.shape

In [ ]:
sns.countplot(y=data_fraud.category)
plt.show()

In [ ]:
plt.figure(figsize=[12,6])
plt.subplot(121)
sns.kdeplot(x=data_fraud[data_fraud.category == "grocery_pos"].Time)
plt.xlim(left=0,right=24)
plt.title(label = "Offline Grocery Transactions Frauds", fontdict = {"color": "Red", "size": 15, "weight" : "5"})
plt.subplot(122)
sns.kdeplot(x=data_fraud[data_fraud.category == "shopping_net"].Time)
plt.title(label = "Internet Shopping Transactions Frauds", fontdict = {"color": "Green", "size": 15, "weight" : "5"})
plt.xlim(left=0,right=24)
plt.show()


In [ ]:
plt.figure(figsize=[12,10])
plt.subplot(121)
sns.countplot(y=data_fraud[data_fraud.category == "grocery_pos"].state)
plt.title(label = "Offline Grocery Transactions Frauds \n by States", fontdict = {"color": "Red", "size": 15, "weight" : "5"})
plt.subplot(122)
sns.countplot(y=data_fraud[data_fraud.category == "shopping_net"].state)
plt.title(label = "Internet Shopping Transactions Frauds \n by States", fontdict = {"color": "Green", "size": 15, "weight" : "5"})
plt.show()

In [ ]:
# let's check how far frauds happens from customer's residence
sns.histplot(data_fraud.location)
plt.show()

In [ ]:
sns.histplot(data_fraud['gender'])
plt.show()

In [ ]:
plt.figure(figsize = [15,10])
sns.countplot(y=data_fraud.state, hue=data_fraud.gender)
plt.show()

In [ ]:
sns.kdeplot(data_fraud.amt)
plt.xlim(left=0)
plt.show()

In [ ]:
# Let's see in which category the losses are huge
data_cat = data_fraud.groupby(by="category")["amt"].sum()
sns.barplot(y=data_cat.index, x=data_cat)
plt.show()


In [ ]:
data_fraud.info()

In [ ]:
data_cat = data_fraud.groupby(by=["category", "is_fraud"])["amt"].sum().reset_index()

# Filter only the fraud entries
data_fraud_only = data_cat[data_cat["is_fraud"] == True]

sns.barplot(y=data_fraud_only["category"], x=data_fraud_only["amt"])
plt.show()

# Creating a DataFrame for Ticket Size
data_ticket = pd.DataFrame({"Category": data_fraud_only["category"], "Ticket Size": data_fraud_only["amt"]})
data_ticket.sort_values(by="Ticket Size", ascending=False, inplace=True)
data_ticket.reset_index(drop=True, inplace=True)

data_ticket


# Model Building 

**Logistic Regression**

**Building a model***

In [ ]:
data4 = data.drop(columns=["trans_date_trans_time","cc_num","merchant","job","state"])
data5 = pd.get_dummies(data=data4,columns=["category","gender","recency_segment","city_pop_segment","location"], drop_first=True)

# assigning them to test dataset

data6 = data_t.drop(columns=["trans_date_trans_time","cc_num","merchant","job","state"])
data_test = pd.get_dummies(data=data6,columns=["category","gender","recency_segment","city_pop_segment","location"], drop_first=True)

In [ ]:
data_test.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

sc=StandardScaler()

In [ ]:
data5[["amt","Time","displacement","recency"]] = sc.fit_transform(data5[["amt","Time","displacement","recency"]])

In [ ]:
data5.head()

In [ ]:
y= data5.is_fraud[:]
x=data5.drop("is_fraud",axis=1)


In [ ]:
data5.info()

In [ ]:
boolean_columns = data5.select_dtypes(include=bool).columns
data5[boolean_columns] = data5[boolean_columns].astype(int)

In [ ]:
data5.info()

In [ ]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# Assuming data5 is your DataFrame
scaler = StandardScaler()
numeric_cols = ["amt", "Time", "recency", "displacement"]
data5[numeric_cols] = scaler.fit_transform(data5[numeric_cols])

# Convert boolean columns to integers
boolean_columns = data5.select_dtypes(include=bool).columns
data5[boolean_columns] = data5[boolean_columns].astype(int)

# Set up target variable y and features x
y = data5["is_fraud"]
x = data5.drop("is_fraud", axis=1)

# Add a constant term to the independent variables
X = sm.add_constant(x)

# Fit the logistic regression model
model = sm.GLM(y, X, family=sm.families.Binomial())
result = model.fit()

# Display the summary
print(result.summary())


# Random forest regressor

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train = data5[:int(len(data5)*0.8)]
test = data5[int(len(data5)*0.8):]
train.shape, test.shape

In [ ]:
target = 'is_fraud'

X_train = train.drop(target, axis=1)
y_train = train[target]
X_test = test.drop(target, axis=1)
y_test = test[target]

X_train.shape, y_train.shape

**Oversampling data to compensate for the high class imbalance in the data**

In [ ]:

# Using ADASYN to oversample

from imblearn.over_sampling import ADASYN
X_train_res, y_train_res = ADASYN().fit_resample(X_train, y_train)

from collections import Counter
print(sorted(Counter(y_train_res).items()))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import GridSearchCV

In [ ]:
#data5 is entire train data set
rf = RandomForestClassifier(n_estimators=10, max_depth=5, max_features=12, min_samples_leaf=100, random_state=100, oob_score=True)


In [ ]:
rf.fit(x,y)

In [ ]:
rf.oob_score_

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Assuming you have a trained model 'rf_model' and test data 'X_test', 'y_test'
# rf_model = ...

# Get predicted probabilities for the positive class
y_probs = rf.predict_proba(X_train_res)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_train_res, y_probs)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_train.columns,
    "Imp": rf.feature_importances_
})

In [ ]:

imp_df.sort_values(by="Imp", ascending=False)

In [ ]:
y_train_pred = rf.predict(X_train_res)
y_train_pred[:10]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train_res, y_train_pred))

**Predicting on test set**

In [ ]:
y_test_pred = rf.predict(X_test)
y_test_pred[:10]

In [ ]:

print(classification_report(y_test, y_test_pred))

**Testing model on original data without oversampling**

In [ ]:
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

**ROC curve on train data set without oversampling**

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Assuming you have a trained model 'rf_model' and test data 'X_test', 'y_test'
# rf_model = ...

# Get predicted probabilities for the positive class
y_probs = rf.predict_proba(X_train)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_train, y_probs)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


****ROC curve on test data set without oversampling****

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Assuming you have a trained model 'rf_model' and test data 'X_test', 'y_test'
# rf_model = ...

# Get predicted probabilities for the positive class
y_probs = rf.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


**Output file for final prediction **

**Recursive function elimination**

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(estimator = logreg, n_features_to_select= 22)
rfe_res = rfe.fit(x,y)

In [ ]:
x.columns[rfe_res.support_]

In [ ]:
list(zip(x.columns,rfe.support_,rfe.ranking_))

In [ ]:
cols = x.columns[rfe_res.support_]

In [ ]:
model2 = sm.GLM(y,sm.add_constant(x[cols]),family = sm.families.Binomial())
res2 = model2.fit()
res2.summary()

**Variance Inflation factors**

In [ ]:
# importing libraries
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
vif = pd.DataFrame({"Features":cols})
vif["VIF"] = [variance_inflation_factor(x[cols].values,i) for i in range(x[cols].shape[1])]
vif = vif.sort_values(by="VIF",ascending = False)

In [ ]:
vif

In [ ]:
pred = res2.predict(sm.add_constant(x[cols]))

In [ ]:
predictions = pd.DataFrame({"Actual": y,"Pred_Prob" : pred})

In [ ]:
predictions.head()

In [ ]:
threshold = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for z in threshold:
  predictions[z] = predictions["Pred_Prob"].map(lambda x: 0 if x<z else 1)

In [ ]:
predictions.head()

**Optimal Value**

In [ ]:
import sklearn.metrics as metrics

In [ ]:
curve = pd.DataFrame(index = threshold, columns = ["accuracy","sensitivity","specificity"])

In [ ]:
for y in threshold:  
  matrix = metrics.confusion_matrix(predictions.Actual,predictions[y])
  accuracy = (matrix[1,1] + matrix[0,0])/ (matrix.sum())
  sensitivity = (matrix[1,1])/(matrix[1,1] + matrix[1,0])
  specificity = (matrix[0,0])/(matrix[0,0] + matrix[0,1])

  curve.loc[y] = [accuracy,sensitivity,specificity]

In [ ]:
curve

In [ ]:
curve.plot()

In [ ]:
for y in threshold:
  curve.loc[y,"Precision"] = metrics.precision_score(predictions.Actual,predictions[y])
  curve.loc[y,"Recall"] = metrics.recall_score(predictions.Actual,predictions[y])

In [ ]:
curve[["Precision","Recall"]].plot()

In [ ]:
curve

**ROC curve**

In [ ]:
# Importing necessary libraries for roc curve

from sklearn.metrics import roc_curve       
from sklearn.metrics import roc_auc_score

# Creating a function to plot roc curve with auc score

def edu_roc( real, probability ):
    
    # Creating roc curve values like false positive rate , true positive rate and threshold
    
    fpr, tpr, thresholds = roc_curve( real, probability,drop_intermediate = False )   
    
    # Calculating the auc score(area under the curve)
    
    auc_score = roc_auc_score( real, probability )    
    
    # Setting the figure size
    
    plt.figure(figsize=(8, 4))
    
    # Plotting the roc curve
    
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    
    # Plotting the 45% dotted line 
    plt.plot([0, 1], [0, 1], 'r--')
    
    # Setting the x axis linit
    
    plt.xlim([0.0, 1.0])
    
    # Setting the y axis limit
    
    plt.ylim([0.0, 1.05])
    
    plt.xlabel('False Positive Rate')
    
    plt.ylabel('True Positive Rate')
    
    plt.title('ROC Curve')
    
    plt.legend(loc="best")

    plt.show()

    return None 

In [ ]:
edu_roc(predictions["Actual"],predictions["Pred_Prob"])

**Test data set**

In [ ]:

x_test = data_test

In [ ]:
# Normalising the data
x_test[["amt","Time","displacement","recency"]] = sc.fit_transform(x_test[["amt","Time","displacement","recency"]])

In [ ]:
boolean_columns = x_test.select_dtypes(include=bool).columns
x_test[boolean_columns] = x_test[boolean_columns].astype(int)

In [ ]:
x_test[cols].dtypes

In [ ]:
result = res2.predict(sm.add_constant(x_test[cols]))

In [ ]:
predictions_test = pd.DataFrame({"Pred_Prob" : result})

In [ ]:
predictions_test["Final"] = predictions_test["Pred_Prob"].map(lambda x: 0 if x<0.01 else 1)

In [ ]:
predictions_test = predictions_test.rename_axis('id').reset_index()

In [ ]:
predictions_test


In [ ]:
predictions_test = predictions_test.set_index('id')

In [ ]:
predictions_test.index = predictions_test.index + 1

In [ ]:
predictions_test=predictions_test.drop("Pred_Prob",axis=1)

In [ ]:
predictions_test.rename(columns={'Final': 'is_fraud'}, inplace=True)

In [ ]:
predictions_test


In [ ]:
predictions_test

In [ ]:
predictions_test.to_csv('/kaggle/working/output_final.csv')

In [ ]:
predictions_test